<a href="https://colab.research.google.com/github/jlsgodev/dados/blob/main/Bucket_Colab_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 38.0 MB/s eta 0:00:00


In [ ]:
!pip install pandas sqlalchemy pymysql


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error



In [ ]:
# Criar um DataFrame de exemplo, sem a coluna auto incrementada
data = {
    'nome': ['Alice', 'Bob', 'Charlie'],
    'salario': [5000.00, 6000.00, 7000.00],
    'numdepto': [5, 2, 3]
}

df = pd.DataFrame(data)
print(df)


      nome  salario  numdepto
0    Alice   5000.0         5
1      Bob   6000.0         2
2  Charlie   7000.0         3


In [ ]:
# Definir as credenciais e informações da conexão
servidor = '35.192.98.101'
basededados = 'EmpresaDB'
usuario = 'adriano'
senha = '@nN32012'

# Função para conectar ao banco de dados
def conectar_mysql(servidor, basededados, usuario, senha):
    try:
        conn = mysql.connector.connect(
            host=servidor,
            database=basededados,
            user=usuario,
            password=senha
        )
        if conn.is_connected():
            print("Conexão bem-sucedida ao banco de dados MySQL")
            return conn
    except Error as e:
        print(f"Erro ao conectar ao MySQL: {e}")
        return None

# Estabelecer a conexão
conexao = conectar_mysql(servidor, basededados, usuario, senha)


Conexão bem-sucedida ao banco de dados MySQL


In [ ]:
# Função para criar a tabela, se necessário
def criar_tabela(conexao):
    try:
        cursor = conexao.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS EMPREGADO (
                matricula INT AUTO_INCREMENT PRIMARY KEY,
                nome VARCHAR(255) NOT NULL,
                salario DECIMAL(10, 2),
                numdepto INT
            )
        """)
        conexao.commit()
        print("Tabela criada ou já existente.")
    except Error as e:
        print(f"Erro ao criar tabela: {e}")

# Criar a tabela
criar_tabela(conexao)


Tabela criada ou já existente.


In [ ]:
# Função para inserir dados do DataFrame
def inserir_dados(df, conexao):
    try:
        cursor = conexao.cursor()
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT INTO EMPREGADO (nome, salario, numdepto)
                VALUES (%s, %s, %s)
            """, (row['nome'], row['salario'], row['numdepto']))
        conexao.commit()
        print("Dados inseridos com sucesso.")
    except Error as e:
        print(f"Erro ao inserir dados: {e}")

# Inserir dados
inserir_dados(df, conexao)


Dados inseridos com sucesso.


In [ ]:
# Função para consultar dados
def consultar_dados(conexao):
    try:
        query = "SELECT * FROM EMPREGADO"
        df_resultado = pd.read_sql(query, con=conexao)
        print(df_resultado)
    except Error as e:
        print(f"Erro ao consultar dados: {e}")

# Consultar e imprimir os dados
consultar_dados(conexao)


    matricula               nome  salario  numdepto
0           2        João Santos   5500.0         2
1           3     Maria Oliveira   6000.0         2
2           4       Carlos Souza   4000.0         3
3           5      Fernanda Lima   4700.0         4
4           6              Diego   7000.0         5
5           8  Dienyffer Cardoso   8500.0         1
6          11            ADRIANO  15000.0         5
7          12            Alberes  10000.0         2
8          13          Jaqueline  15000.0         3
9          14              Helen   9000.0         4
10         15              Alice   5000.0         5
11         16                Bob   6000.0         2
12         17            Charlie   7000.0         3


<ipython-input-14-c24e179fee37>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultado = pd.read_sql(query, con=conexao)


In [ ]:
# Instalar bibliotecas
!pip install pandas google-cloud-storage mysql-connector-python

# Importar bibliotecas
import pandas as pd
from google.cloud import storage
import mysql.connector
from mysql.connector import Error

# Autenticar a conta do Google Cloud
from google.colab import auth
auth.authenticate_user()

# Configurar o cliente do Google Cloud Storage
client = storage.Client()
bucket_name = 'minhasBases'
file_path = '/path/to/your/local/file.csv'

# Função para fazer upload de arquivos para o bucket GCS
def upload_to_gcs(bucket_name, file_path):
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob('dados_brutos/arquivo.csv')
    blob.upload_from_filename(file_path)
    print(f'Arquivo {file_path} carregado para {bucket_name}/dados_brutos/arquivo.csv')

upload_to_gcs(bucket_name, file_path)

# Função para ler o arquivo CSV do GCS para um DataFrame
def read_csv_from_gcs(bucket_name, file_name):
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    blob.download_to_filename(file_name)
    df = pd.read_csv(file_name)
    return df

df = read_csv_from_gcs(bucket_name, 'dados_brutos/arquivo.csv')
print(df.head())

# Definir as credenciais e informações da conexão
servidor = 'seuIP'
basededados = 'seuprojeto01'
usuario = 'seuusuario'
senha = 'suasenha'

# Função para conectar ao banco de dados
def conectar_mysql(servidor, basededados, usuario, senha):
    try:
        conn = mysql.connector.connect(
            host=servidor,
            user=usuario,
            password=senha
        )
        if conn.is_connected():
            print("Conexão bem-sucedida ao MySQL")
            return conn
    except Error as e:
        print(f"Erro ao conectar ao MySQL: {e}")
        return None

conexao = conectar_mysql(servidor, basededados, usuario, senha)

# Função para criar o banco de dados
def criar_banco(conexao, basededados):
    try:
        cursor = conexao.cursor()
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {basededados}")
        conexao.commit()
        print(f"Banco de dados {basededados} criado ou já existente.")
    except Error as e:
        print(f"Erro ao criar banco de dados: {e}")

criar_banco(conexao, basededados)

# Função para criar a tabela com base no DataFrame
def criar_tabela(conexao, df):
    cursor = conexao.cursor()
    columns = ', '.join(f"{col} VARCHAR(255)" for col in df.columns)
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS tabela_dados (
        id INT AUTO_INCREMENT PRIMARY KEY,
        {columns}
    )
    """
    try:
        cursor.execute(create_table_query)
        conexao.commit()
        print("Tabela criada com sucesso.")
    except Error as e:
        print(f"Erro ao criar tabela: {e}")

criar_tabela(conexao, df)

# Função para inserir dados do DataFrame na tabela
def inserir_dados(df, conexao):
    cursor = conexao.cursor()
    for _, row in df.iterrows():
        values = ', '.join(f"'{str(val)}'" for val in row)
        insert_query = f"INSERT INTO tabela_dados ({', '.join(df.columns)}) VALUES ({values})"
        try:
            cursor.execute(insert_query)
        except Error as e:
            print(f"Erro ao inserir dados: {e}")
    conexao.commit()
    print("Dados inseridos com sucesso.")

inserir_dados(df, conexao)

# Função para consultar dados
def consultar_dados(conexao):
    try:
        query = "SELECT * FROM tabela_dados"
        df_resultado = pd.read_sql(query, con=conexao)
        print(df_resultado)
    except Error as e:
        print(f"Erro ao consultar dados: {e}")

consultar_dados(conexao)

# Fechar a conexão
if conexao.is_connected():
    conexao.close()
    print("Conexão fechada.")
